In [211]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fuzzywuzzy import fuzz

In [212]:
# read in the dataset
data = pd.read_csv("Surveys.csv", sep=',').replace('"', '', regex=True)

In [213]:
# display data to get a grasp on what the whole dataset looks like
display(data.head())

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


# Data explanation
The dataset consists of survey data from 2021 and 2022 regarding people's salary.<br/>
It containts 27922 entries and is made out of 18 variables such as salary, job title, industry, etc.<br/>
The column names are the questions given to the respondent.<br/>
The survey data can retrieved from https://oscarbaruffa.com/messy/.<br/>
The survey form can be retrieved from https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html.<br/>

The following variables are present in the dataset.<br/>

## Timestamp
- The datetime at which the respondent submitted their entry.<br/>
- This variable was generated by the software handling the survey data<br/>
- This variable is never emtpy.<br/>

## Age band
- The age band in which the resondent belongs to.<br/>
- The respondent chose from a premade list of answers.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can be chosen.<br/>

## Industry
- The industry in which the respondent works.<br/>
- The respondent either chose from a premade list of answers or gave their own answer.<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be chosen or given.<br/>

## Job title
- The job title of the respondent.<br/>
- The respondent gave their own answer.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can given.<br/>

## Job context
- Additional context regarding the respondent's job title.<br/>
- The respondent gave their own answer.<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be given.<br/>

## Salary
- The respondent's annual salary based on 40 hours a week, 52 weeks a year.<br/>
- The respondent gave their own answer.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can be given.<br/>

## Compensation
- Additional monetary income if the respondent has any.<br/>
- The respondent gave their own answer.<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be given.<br/>

## Currency
- The currency in which the respondent receives their salary abd compensation.<br/>
- The respondent chose from a premade list of answers.<br/>
- This variable is mandatory ('Other' is an answer in the list).<br/>
- Only 1 answer can be chosen.<br/>

## Other currency
- The respondent's currency in case it wasn't an option in the premade list.<br/>
- The respondent gave their own answer.<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be given.<br/>

## Income context
- Additional context regarding the salary and compensation of the respondent.<br/>
- The respondent gave their own answer.<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be given.<br/>

## Country
- The country in which the respondent works.<br/>
- The respondent gave their own answer.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can be given.<br/>

## State
- The state or states of the respondent in case the respondent works in the USA.<br/>
- The respondent chose from a premade list of answers.<br/>
- This variable is not mandatory.<br/>
- Multiple answers could be chosen.<br/>

## City
- The city in which the respondent works.<br/>
- The respondent gave their own answer.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can be given.<br/>

## Overall professional years of eperience band
- The band in which the respondent has overall professional years of experience.<br/>
- The respondent chose from a premade list of answers.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can be chosen.<br/>

## Field professional years of eperience band
- The band in which the respondent professional years of experience in their current field.<br/>
- The respondent chose from a premade list of answers.<br/>
- This variable is mandatory.<br/>
- Only 1 answer can be chosen.<br/>

## Education
- The respondent's highest level of education.<br/>
- The respondent chose from a premade list of answers.<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be chosen.<br/>

## Gender
- The respondent's gender.<br/>
- The respondent chose from a premade list of answers ("Other/no answer" is an answer in the list).<br/>
- This variable is not mandatory.<br/>
- Only 1 answer could be chosen.<br/>

## Race
- The respondent's race or races.<br/>
- The respondent chose from a premade list of answers ("Other/no answer" is an answer in the list).<br/>
- This variable is not mandatory.<br/>
- Multiple answers could be chosen.<br/>

# Data cleansing and preparation

In [214]:
# new column names
new_columns = [
    "datetime", "age_band", "industry", "job_title", "job_context", "salary", "compensation", "currency", \
    "other_currency", "income_context", "country", "state", "city", "overall_experience_band", "field_experience_band", \
    "education", "gender", "race"
]

# map old column names to new ones
mapping = {}
for i in range(0, len(data.columns)):
    mapping.update({data.columns[i]: new_columns[i]})
    
# rename columns
data = data.rename(mapping, axis='columns')

# check if columns are renamed
display(data.head())

,datetime,age_band,industry,job_title,job_context,salary,compensation,currency,other_currency,income_context,country,state,city,overall_experience_band,field_experience_band,education,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [215]:
# check what kind of dtype each column has
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27922 entries, 0 to 27921
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   datetime                 27922 non-null  object 
 1   age_band                 27922 non-null  object 
 2   industry                 27850 non-null  object 
 3   job_title                27922 non-null  object 
 4   job_context              7226 non-null   object 
 5   salary                   27922 non-null  object 
 6   compensation             20677 non-null  float64
 7   currency                 27922 non-null  object 
 8   other_currency           196 non-null    object 
 9   income_context           3033 non-null   object 
 10  country                  27922 non-null  object 
 11  state                    22945 non-null  object 
 12  city                     27847 non-null  object 
 13  overall_experience_band  27922 non-null  object 
 14  field_experience_band 

In [216]:
# change datetime format and dtype
data['datetime'] = pd.to_datetime(data['datetime'], infer_datetime_format=True)

# check if dtype and values are correct
print(data['datetime'].info())
display(data[['datetime']].head())

<class 'pandas.core.series.Series'>
RangeIndex: 27922 entries, 0 to 27921
Series name: datetime
Non-Null Count  Dtype         
--------------  -----         
27922 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 218.3 KB
None


,datetime
0,2021-04-27 11:02:10
1,2021-04-27 11:02:22
2,2021-04-27 11:02:38
3,2021-04-27 11:02:41
4,2021-04-27 11:02:42


In [217]:
# change age band dtype
data['age_band'] = data['age_band'].astype('category')

# change industry dtype
data['industry'] = data['industry'].astype('category')

# check if dtype got changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27922 entries, 0 to 27921
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27922 non-null  datetime64[ns]
 1   age_band                 27922 non-null  category      
 2   industry                 27850 non-null  category      
 3   job_title                27922 non-null  object        
 4   job_context              7226 non-null   object        
 5   salary                   27922 non-null  object        
 6   compensation             20677 non-null  float64       
 7   currency                 27922 non-null  object        
 8   other_currency           196 non-null    object        
 9   income_context           3033 non-null   object        
 10  country                  27922 non-null  object        
 11  state                    22945 non-null  object        
 12  city                     27847 n

In [218]:
# check how many NaN industry has
print(data['industry'].isnull().sum())
print(data.shape)

# only take rows with non-NaN
data = data[data['industry'].notna()]

# check if rows are removed
print(data['industry'].isnull().sum())
print(data.shape)

72
(27922, 18)
0
(27850, 18)


In [219]:
# https://towardsdatascience.com/string-matching-with-fuzzywuzzy-e982c61f8a84
# https://pypi.org/project/fuzzywuzzy/

# replace 'Health care' with 'Health Care'
data['industry'] = data['industry'].replace('Health care', 'Health Care')

# all answers that could be chosen for industry
premade_catgs = [
    'Accounting, Banking & Finance',
    'Agriculture or Forestry',
    'Art & Design',
    'Business or Consulting',
    'Computing or Tech',
    'Education (Primary/Secondary)',
    'Education (Higher Education)',
    'Engineering or Manufacturing',
    'Entertainment',
    'Government and Public Administration',
    'Health Care',
    'Hospitality & Events',
    'Insurance',
    'Law',
    'Law Enforcement & Security',
    'Leisure, Sport & Tourism',
    'Marketing, Advertising & PR',
    'Media & Digital',
    'Nonprofits',
    'Property or Construction',
    'Recruitment or HR',
    'Retail',
    'Sales',
    'Social Work',
    'Transport or Logistics',
    'Utilities & Telecommunications'
]

# adding this categories manually a good amount of answers involving it
premade_catgs.append('Library or Archiving')

# all answers present in 'industry'
all_catgs = list(data['industry'])

# get all text-input answers
input_catgs = []
for catg in all_catgs:
    if catg not in premade_catgs:
        input_catgs.append(catg)

# map text-input answers to a premade answer
catg_mapping = {}
catg_ratio_threshold = 75
for input_catg in input_catgs:
    matches = []
    for premade_catg in premade_catgs:
        
        # try to match every text-input answers to a premade answer when a threshold is met
        TSR_score = fuzz.token_set_ratio(input_catg, premade_catg)
        PR_score = fuzz.partial_ratio(input_catg.lower(), premade_catg.lower())
        if TSR_score >= catg_ratio_threshold:
            matches.append(tuple([input_catg, premade_catg, TSR_score]))
        if PR_score >= catg_ratio_threshold:
            matches.append(tuple([input_catg, premade_catg, PR_score])) 
            
    # if a text-input answer has multiple mactches, pick the one with the highest score
    if len(matches) > 0:
        best_match = matches[0]
        for match in matches:
            if match[2] > best_match[2]:
                best_match = match
        catg_mapping.update({best_match[0]: best_match[1]})
                
    # if no match is found, text-input answer is mapped to 'Other'
    else:
        catg_mapping.update({input_catg: "Other"})

In [220]:
# replace text-input answers for industry
data['industry'] = data['industry'].replace(catg_mapping)

# check categories present in industry
print(data['industry'].value_counts())

Computing or Tech                       4671
Education (Higher Education)            2465
Nonprofits                              2434
Health Care                             1899
Government and Public Administration    1897
Accounting, Banking & Finance           1798
Other                                   1794
Engineering or Manufacturing            1746
Marketing, Advertising & PR             1123
Law                                     1097
Business or Consulting                   861
Education (Primary/Secondary)            836
Media & Digital                          773
Insurance                                534
Retail                                   509
Recruitment or HR                        458
Property or Construction                 400
Utilities & Telecommunications           374
Art & Design                             365
Sales                                    353
Transport or Logistics                   316
Social Work                              273
Hospitalit

In [221]:
# check how many unique job titles there are before lowercasing
print(len(data['job_title'].unique()))

# change job titles to lowercase
data['job_title'] = data['job_title'].str.lower()

# check how many unique job titles there are after lowercasing
print(len(data['job_title'].unique()))

14249
13005


In [225]:
# check how many NaN job context has
print(data['job_context'].isnull().sum())

# drop job context, too many NaN's
data = data.drop('job_context', axis='columns')

# check if job context got removed
print(data.shape)

20636
(27850, 17)


In [226]:
# remove american notation
data['salary'] = data['salary'].str.replace(',', '')

# change salary dtype
data['salary'] = data['salary'].astype(int)

# check if dtype got changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27850 entries, 0 to 27921
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27850 non-null  datetime64[ns]
 1   age_band                 27850 non-null  category      
 2   industry                 27850 non-null  category      
 3   job_title                27850 non-null  object        
 4   salary                   27850 non-null  int64         
 5   compensation             20629 non-null  float64       
 6   currency                 27850 non-null  object        
 7   other_currency           195 non-null    object        
 8   income_context           3031 non-null   object        
 9   country                  27850 non-null  object        
 10  state                    22889 non-null  object        
 11  city                     27775 non-null  object        
 12  overall_experience_band  27850 n

In [227]:
# check which values are in other currency
print(data['other_currency'].value_counts())

USD                                                                                                                 11
INR                                                                                                                 10
NOK                                                                                                                 10
SGD                                                                                                                  9
MYR                                                                                                                  8
DKK                                                                                                                  8
AUD                                                                                                                  7
BRL                                                                                                                  6
PLN                                             

In [228]:
# check how many times 'Other' appears in currency
print(data['currency'].value_counts())

USD        23210
CAD         1660
GBP         1581
EUR          633
AUD/NZD      498
Other        154
CHF           37
SEK           37
JPY           23
ZAR           13
HKD            4
Name: currency, dtype: int64


In [229]:
# drop currency with 'Other' value, too insignificant
print(data.shape)
data = data.drop(data[data['currency'] == 'Other'].index)

# drop other currency, too many NaN's
data = data.drop('other_currency', axis='columns')

# check if rows and column got removed
print(data.shape)

(27850, 17)
(27696, 16)


In [230]:
# fill compensation with value '0' if NaN
data['compensation'] = data['compensation'].fillna(0)

# change compensation dtype
data['compensation'] = data['compensation'].astype(int)

# check if dtype changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27696 entries, 0 to 27921
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27696 non-null  datetime64[ns]
 1   age_band                 27696 non-null  category      
 2   industry                 27696 non-null  category      
 3   job_title                27696 non-null  object        
 4   salary                   27696 non-null  int64         
 5   compensation             27696 non-null  int64         
 6   currency                 27696 non-null  object        
 7   income_context           3008 non-null   object        
 8   country                  27696 non-null  object        
 9   state                    22876 non-null  object        
 10  city                     27621 non-null  object        
 11  overall_experience_band  27696 non-null  object        
 12  field_experience_band    27696 n

In [231]:
# check how many NaN income context has
print(data['income_context'].isnull().sum())

# drop income context, too many NaN's
data = data.drop('income_context', axis='columns')

# check if job context got removed
print(data.shape)

24688
(27696, 15)


In [232]:
display(data.head())

,datetime,age_band,industry,job_title,salary,compensation,currency,country,state,city,overall_experience_band,field_experience_band,education,gender,race
0,2021-04-27 11:02:10,25-34,Education (Higher Education),research and instruction librarian,55000,0,USD,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,2021-04-27 11:02:22,25-34,Computing or Tech,change & internal communications manager,54600,4000,GBP,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,2021-04-27 11:02:38,25-34,"Accounting, Banking & Finance",marketing specialist,34000,0,USD,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,2021-04-27 11:02:41,25-34,Nonprofits,program manager,62000,3000,USD,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,2021-04-27 11:02:42,25-34,"Accounting, Banking & Finance",accounting manager,60000,7000,USD,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


In [233]:
# change country to lowercase
data['country'] = data['country'].str.lower()

# remove punctuation
data['country'] = data['country'].str.replace('.', '')

# check which values are present in country
print(data['country'].value_counts())

united states                                                                                                                                                                                                        9275
usa                                                                                                                                                                                                                  8568
us                                                                                                                                                                                                                   3333
canada                                                                                                                                                                                                               1588
united states                                                                                                                   

/tmp/ipykernel_41508/1744000476.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['country'] = data['country'].str.replace('.', '')


In [234]:
# list with correct values
correct_countries = [
    'united states of america',
    'united states',
    'usa',
    'united kingdom',
    'great britain'
    'uk',
    'england',
    'canada',
    'germany',
    'france',
    'spain',
    'scotland',
    'netherlands',
    'australia',
    'austria',
    'new zealand',
    'argentina',
    'italy',
    'finland',
    'wales',
    'ireland',
    'belgium',
    'switzerland',
    'japan',
    'south africa',
    'denmark'
]

input_countries = list(data['country'])

# map incorrect country to correct country
country_mapping = {}
country_ratio_threshold = 75
for input_country in input_countries:
    matches = []
    for correct_country in correct_countries:
        
        # try to match every text-input answers to a correct country variation when a threshold is met
        TSR_score = fuzz.token_set_ratio(input_country, correct_country)
        if TSR_score >= country_ratio_threshold:
            matches.append(tuple([input_country, correct_country, TSR_score]))
            
    # if a text-input answer has multiple mactches, pick the one with the highest score
    if len(matches) > 0:
        best_match = matches[0]
        for match in matches:
            if match[2] > best_match[2]:
                best_match = match
        country_mapping.update({best_match[0]: best_match[1]})
        
    # if no match is found, text-input answer is mapped to 'unknown'
    else:
        country_mapping.update({input_catg: "unknown"})

In [235]:
# replace text-input answers for country
data['country'] = data['country'].replace(country_mapping)

# remove trailing whitespaces
data['country'] = data['country'].str.strip()

# check categories present in country
print(data['country'].value_counts())

usa                                                                                                                                                                                                                  12439
united states of america                                                                                                                                                                                             10464
canada                                                                                                                                                                                                                1673
uk                                                                                                                                                                                                                     700
united kingdom                                                                                                              

In [236]:
# improve certain mappings
improved_country_mapping = {
    'usa': 'united states of america',
    'united states': 'united states of america',
    'uk': 'united kingdom',
    'england': 'united kingdom',
    'great britain': 'united kingdom',
    'scotland': 'united kingdom',
    'wales': 'united kingdom'
}

# refine mappings for country
data['country'] = data['country'].replace(improved_country_mapping)

# check categories present in country
print(data['country'].value_counts())

united states of america                                                                                                                                                                                             22996
canada                                                                                                                                                                                                                1673
united kingdom                                                                                                                                                                                                        1573
australia                                                                                                                                                                                                              381
germany                                                                                                                     

In [237]:
# create df of correct countries
correct_countries_df = pd.DataFrame(correct_countries).rename({0: 'country'}, axis='columns')
display(correct_countries_df.head())

,country
0,united states of america
1,united states
2,usa
3,united kingdom
4,great britainuk


In [238]:
# create filter to remove remaining countries
print(data.shape)
is_correct_country = data['country'].isin(correct_countries_df['country'])
data = data.drop(data[~is_correct_country].index)

# check if rows got dropped
print(data.shape)

(27696, 15)
(27450, 15)


In [239]:
# change country dtype
data['country'] = data['country'].astype('category')

# check if dtype got changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27450 entries, 0 to 27921
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27450 non-null  datetime64[ns]
 1   age_band                 27450 non-null  category      
 2   industry                 27450 non-null  category      
 3   job_title                27450 non-null  object        
 4   salary                   27450 non-null  int64         
 5   compensation             27450 non-null  int64         
 6   currency                 27450 non-null  object        
 7   country                  27450 non-null  category      
 8   state                    22831 non-null  object        
 9   city                     27376 non-null  object        
 10  overall_experience_band  27450 non-null  object        
 11  field_experience_band    27450 non-null  object        
 12  education                27248 n

In [240]:
# create filter to drop 'united states of america' with no state
print(data.shape)
usa_no_state = (data['country'] == 'united states of america') & (data['state'].isna())

# drop rows
data = data.drop(data[usa_no_state].index)

# check if rows got dropped
print(data.shape)

(27450, 15)
(27280, 15)


In [241]:
# fill state NaN with value 'Not American'
data['state'] = data['state'].fillna('Not American')

# check if NaN still exists
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27280 entries, 0 to 27921
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27280 non-null  datetime64[ns]
 1   age_band                 27280 non-null  category      
 2   industry                 27280 non-null  category      
 3   job_title                27280 non-null  object        
 4   salary                   27280 non-null  int64         
 5   compensation             27280 non-null  int64         
 6   currency                 27280 non-null  object        
 7   country                  27280 non-null  category      
 8   state                    27280 non-null  object        
 9   city                     27210 non-null  object        
 10  overall_experience_band  27280 non-null  object        
 11  field_experience_band    27280 non-null  object        
 12  education                27090 n

In [242]:
# change country dtype to category
data['country'] = data['country'].astype('category')

# change state dtype to category
data['state'] = data['state'].astype('category')

# check if dtypes got changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27280 entries, 0 to 27921
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27280 non-null  datetime64[ns]
 1   age_band                 27280 non-null  category      
 2   industry                 27280 non-null  category      
 3   job_title                27280 non-null  object        
 4   salary                   27280 non-null  int64         
 5   compensation             27280 non-null  int64         
 6   currency                 27280 non-null  object        
 7   country                  27280 non-null  category      
 8   state                    27280 non-null  category      
 9   city                     27210 non-null  object        
 10  overall_experience_band  27280 non-null  object        
 11  field_experience_band    27280 non-null  object        
 12  education                27090 n

In [243]:
# drop city column (unable to clean without extensive effort)
print(data.shape)
data = data.drop('city', axis='columns')

# check if column got dropped
print(data.shape)

(27280, 15)
(27280, 14)


In [244]:
# fix small typo in overall experience band and field experience band
data['overall_experience_band'] = data['overall_experience_band'].str.replace(' - ', '-')
data['field_experience_band'] = data['field_experience_band'].str.replace(' - ', '-')

# check if typos got fixed
print(data['overall_experience_band'].value_counts(), '\n')
print(data['overall_experience_band'].value_counts())

11-20 years         9380
8-10 years          5264
5-7 years           4739
21-30 years         3547
2-4 years           2892
31-40 years          846
1 year or less       493
41 years or more     119
Name: overall_experience_band, dtype: int64 

11-20 years         9380
8-10 years          5264
5-7 years           4739
21-30 years         3547
2-4 years           2892
31-40 years          846
1 year or less       493
41 years or more     119
Name: overall_experience_band, dtype: int64


In [245]:
# change overall experience band and field experience band to category
data['overall_experience_band'] = data['overall_experience_band'].astype('category')
data['field_experience_band'] = data['field_experience_band'].astype('category')

# check if dtypes got changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27280 entries, 0 to 27921
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27280 non-null  datetime64[ns]
 1   age_band                 27280 non-null  category      
 2   industry                 27280 non-null  category      
 3   job_title                27280 non-null  object        
 4   salary                   27280 non-null  int64         
 5   compensation             27280 non-null  int64         
 6   currency                 27280 non-null  object        
 7   country                  27280 non-null  category      
 8   state                    27280 non-null  category      
 9   overall_experience_band  27280 non-null  category      
 10  field_experience_band    27280 non-null  category      
 11  education                27090 non-null  object        
 12  gender                   27127 n

In [249]:
# check values for education
print(data['education'].value_counts())

College degree                        13178
Master's degree                        8630
Some college                           1995
PhD                                    1383
Professional degree (MD, JD, etc.)     1295
High School                             609
Name: education, dtype: int64


In [248]:
# check how many NaN's education has
print(data.shape)
print(data['education'].isnull().sum())

# drop rows with NaN's (no viable way to fill)
data = data[data['education'].notna()]

# check if rows got dropped
print(data.shape)

(27280, 14)
190
(27090, 14)


In [250]:
# change education dtype
data['education'] = data['education'].astype('category')

# check if dtype has changed
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27090 entries, 0 to 27921
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27090 non-null  datetime64[ns]
 1   age_band                 27090 non-null  category      
 2   industry                 27090 non-null  category      
 3   job_title                27090 non-null  object        
 4   salary                   27090 non-null  int64         
 5   compensation             27090 non-null  int64         
 6   currency                 27090 non-null  object        
 7   country                  27090 non-null  category      
 8   state                    27090 non-null  category      
 9   overall_experience_band  27090 non-null  category      
 10  field_experience_band    27090 non-null  category      
 11  education                27090 non-null  category      
 12  gender                   26966 n

In [252]:
# check values for gender
print(data['gender'].value_counts())

Woman                            20732
Man                               5225
Non-binary                         734
Other or prefer not to answer      274
Prefer not to answer                 1
Name: gender, dtype: int64


In [254]:
# remap some gender values
gender_mapping = {
    'Other or prefer not to answer': 'No answer',
    'Prefer not to answer': 'No answer'
}

# replace values
data['gender'] = data['gender'].replace(gender_mapping)

# check if values got replaced
print(data['gender'].value_counts())

Woman         20732
Man            5225
Non-binary      734
No answer       275
Name: gender, dtype: int64


In [257]:
# check for NaN's in gender
print(data['gender'].isnull().sum())

# fill NaN's with 'No answer'
data['gender'] = data['gender'].fillna('No answer')

# change gender type
data['gender'] = data['gender'].astype('category')

# check to see if NaN's are gone and dtype is changed
print(data.info())

0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 27090 entries, 0 to 27921
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27090 non-null  datetime64[ns]
 1   age_band                 27090 non-null  category      
 2   industry                 27090 non-null  category      
 3   job_title                27090 non-null  object        
 4   salary                   27090 non-null  int64         
 5   compensation             27090 non-null  int64         
 6   currency                 27090 non-null  object        
 7   country                  27090 non-null  category      
 8   state                    27090 non-null  category      
 9   overall_experience_band  27090 non-null  category      
 10  field_experience_band    27090 non-null  category      
 11  education                27090 non-null  category      
 12  gender                   27090

In [261]:
# drop race (prefered to not use this data for predictive modeling)
data = data.drop('race', axis='columns')

# check if column got dropped
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27090 entries, 0 to 27921
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetime                 27090 non-null  datetime64[ns]
 1   age_band                 27090 non-null  category      
 2   industry                 27090 non-null  category      
 3   job_title                27090 non-null  object        
 4   salary                   27090 non-null  int64         
 5   compensation             27090 non-null  int64         
 6   currency                 27090 non-null  object        
 7   country                  27090 non-null  category      
 8   state                    27090 non-null  category      
 9   overall_experience_band  27090 non-null  category      
 10  field_experience_band    27090 non-null  category      
 11  education                27090 non-null  category      
 12  gender                   27090 n

In [266]:
# save cleaned data
data.to_csv('Surveys_cleaned.csv', index=False)